In [ ]:
# imports

import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, Conv2D, Conv2DTranspose, MaxPooling2D, UpSampling2D, Flatten, Reshape, BatchNormalization
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import RandomNormal
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Activation, Add, Lambda
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.regularizers import l1, l2,l1_l2
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd


In [ ]:
#THE DATA HAS BEEN COVERTED TO NUMPY ARRAY AND SAVED AT THE FOLLOWING LOCATION
DATA_LOAD_PATH = './data'
MODEL_SAVE_PATH = 'model.h5'


In [ ]:
class Siamese():

    
    def __init__(self, categories, categories_test, MODEL_SAVE_PATH):
        
        
        self.dataset = categories
        self.test = categories_test
        self.MODEL_SAVE_PATH = MODEL_SAVE_PATH
        
        
        def get_siamese_model(self):
            
            
            
            def W_init(shape,dtype=None):
                
                """Initialize weights"""
                
                values = np.random.normal(loc=0,scale=1e-2,size=shape)
                return K.variable(values,dtype=dtype)
            
            def b_init(shape,dtype=None):
                
                """Initialize bias"""
                
                values = np.random.normal(loc=0.5,scale=1e-2,size=shape)
                return K.variable(values,dtype=dtype)
                    
            
            """
                Model architecture
            """
            
            input_shape = (256, 256, 3)
            
            vgg16 = VGG16(weights = "imagenet", include_top=False, input_shape = (256, 256, 3))

            for layer in vgg16.layers:
                print(layer.name)
                layer.trainable = False

            pre = Model(inputs = vgg16.input, outputs= vgg16.output)
            
            # Define the tensors for the two input images
            left_input = Input(input_shape)
            right_input = Input(input_shape)
            
            # Convolutional Neural Network
            model = Sequential()
        
            model.add(pre)     
            
            model.add(Conv2D(512, (3,3), activation='relu', padding='same', kernel_initializer=W_init, bias_initializer=b_init)) #kernel_regularizer=l1(0.001)))
            model.add(Dropout(0.2))
            model.add(MaxPooling2D())
            model.add(BatchNormalization())
            
            model.add(Conv2D(512, (3,3), activation='relu', padding='same', kernel_initializer=W_init, bias_initializer=b_init))#kernel_regularizer=l1(0.001)))
            model.add(Dropout(0.2))
            model.add(MaxPooling2D())
            model.add(BatchNormalization())
            model.add(Flatten())
            #model.add(Dropout(0.2))
            
            model.add(Dense(4096, activation='sigmoid', kernel_initializer=W_init, bias_initializer=b_init
                           #kernel_regularizer=l1(0.001),
                           ))
            model.add(Dropout(0.2))
            model.add(BatchNormalization())
            
            
            # Generate the encodings (feature vectors) for the two images
            encoded_l = model(left_input)
            encoded_r = model(right_input)
            
            # Add a customized layer to compute the absolute difference between the encodings
            L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
            L1_distance = L1_layer([encoded_l, encoded_r])
            
            # Add a dense layer with a sigmoid unit to generate the similarity score
            prediction = Dense(1,activation='sigmoid')(L1_distance)
            
            # Connect the inputs with the outputs
            siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)
            
           
            optimizer = Adam(0.00006)

            siamese_net.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
            
            # return the model
            return siamese_net
            
        self.siamese_net = get_siamese_model(self)
        
        print(self.siamese_net.summary())
        
        print('Siamese Net Intialised and Loaded')
        


    def get_train_set(self):

            x_anchor=[]
            x_examples = []
            y=[]


            for i in range(len(self.dataset)):
                current_class = self.dataset[i]
                for j in range(len(current_class)):
                    y.append(current_class[j][2])
                    x_anchor.append(current_class[j][0])
                    x_examples.append(current_class[j][1])

            x_anchor = np.asarray(x_anchor)
            x_examples = np.asarray(x_examples)
            y_batch = np.asarray(y)


            print(x_anchor.shape)
            print(x_examples.shape)
            print(y_batch.shape)

            return x_anchor, x_examples, y_batch


    def get_test_set(self):

            x_anchor=[]
            x_examples = []
            y=[]

            for i in range(len(self.test)):
                current_class = self.test[i]
                for j in range(len(current_class)):
                    y.append(current_class[j][2])
                    x_anchor.append(current_class[j][0])
                    x_examples.append(current_class[j][1])

            x_anchor = np.asarray(x_anchor)
            x_examples = np.asarray(x_examples)
            y_batch = np.asarray(y)

            print(x_anchor.shape)
            print(x_examples.shape)
            print(y_batch.shape)

            return x_anchor, x_examples, y_batch



    def get_accuracy(self, preds, y_batch_test):

            preds = np.around(preds)
            results = metrics.accuracy_score(y_batch_test, preds)

            return results
        

            
                      
    def train(self):

            x_anchor, x_examples, y_batch = self.get_train_set()

            x_anchor_test, x_examples_test, y_batch_test = self.get_test_set()

            checkpoint = ModelCheckpoint(MODEL_SAVE_PATH, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

            history = self.siamese_net.fit(x = [x_anchor, x_examples], y=y_batch, epochs=512, batch_size= 32,  verbose = 2, validation_split=0.2, callbacks=[checkpoint])

            # summarize history for accuracy
            plt.plot(history.history['accuracy'])
            plt.plot(history.history['val_accuracy'])
            plt.title('Model Accuracy')
            plt.ylabel('Accuracy')
            plt.xlabel('Iteration')
            plt.legend(['Train Accuracy', 'Validation Accuracy'], loc='upper left')
            plt.savefig('/kaggle/working/accs.png')
            plt.show()

            # summarize history for loss
            plt.plot(history.history['loss'])
            plt.plot(history.history['val_loss'])
            plt.title('Model Loss')
            plt.ylabel('Loss')
            plt.xlabel('Iteration')
            plt.legend(['Train Loss', 'Validation Loss'], loc='upper left')
            plt.savefig('/kaggle/working/losses.png')
            plt.show()

            y_pred = self.siamese_net.predict([x_anchor_test, x_examples_test])
            
            accuracy = self.get_accuracy(np.around(y_pred), y_batch_test)
            print('Accuracy on Test Set is:')
            print(accuracy)

            con_mat = confusion_matrix(y_batch_test, np.around(y_pred))

            df_cm = pd.DataFrame(con_mat, range(len(con_mat)), range(len(con_mat)))

            plt.figure(figsize = (10,7))
            sn.heatmap(df_cm, annot=True)

            plt.savefig('./conf_matrix.png')


            return history

In [ ]:
# Load Data
#THE DATA HAS BEEN COVERTED TO NUMPY ARRAY
train = np.load(os.path.join(DATA_LOAD_PATH, 'train_colour.npy'),  allow_pickle = True)
test = np.load(os.path.join(DATA_LOAD_PATH, 'test_colour.npy'),  allow_pickle = True)

# Create Model 
siamese = Siamese(train, test, MODEL_SAVE_PATH)


In [ ]:
# Train and save metrics 
hist = siamese.train()

In [ ]:
def normalise(img_path):
    img = cv2.imread(img_path, 0)
    img = cv2.resize(img, (256,256))
    norm_image = cv2.normalize(img, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    norm_image = np.reshape(norm_image, (1,256,256,3))
    return norm_image

In [ ]:
from tensorflow.keras.models import load_model
import numpy as np
model=load_model('model.h5')